In [1]:
import numpy as np
import pandas as pd
from math import radians, degrees, sin, cos, asin, acos, sqrt

In [2]:
def great_circle(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    return 3958.756 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    )

In [3]:
recruits = pd.read_csv('https://raw.githubusercontent.com/BenDavis71/ImGonnaCroot500Miles/master/recruits-lat-long.csv') 
teams = pd.read_csv('https://raw.githubusercontent.com/BenDavis71/ImGonnaCroot500Miles/master/teams-lat-long.csv')
teamsList = teams['school'].tolist()

In [4]:
#user input for star range
stars = [4,5]

In [5]:
#user input for distance
distance = 250

In [6]:
recruits = recruits[recruits['stars'].between(stars[0],stars[1])]

recruits['merge'] = 1
teams['merge'] = 1
df = pd.merge(recruits,teams,on='merge')

In [7]:
df['distance'] = df.apply(lambda row: great_circle(row['lng_x'], row['lat_x'], row['lng_y'], row['lat_y']), axis = 1)
df = df[df['distance'] <= distance]
df

,year,name,committedTo,position,stars,city,lat_x,lng_x,count,merge,school,color,logos,lat_y,lng_y,distance
33,2010,Ronald Powell,Florida,DE,5,"Moreno Valley, CA",33.9244,-117.2045,1,1,Fresno State,#c41230,['http://a.espncdn.com/i/teamlogos/ncaa/500/27...,36.814320,-119.758102,246.083518
91,2010,Ronald Powell,Florida,DE,5,"Moreno Valley, CA",33.9244,-117.2045,1,1,San Diego State,#BF2C37,['http://a.espncdn.com/i/teamlogos/ncaa/500/21...,32.783053,-117.119601,79.011505
113,2010,Ronald Powell,Florida,DE,5,"Moreno Valley, CA",33.9244,-117.2045,1,1,UCLA,#005C8E,['http://a.espncdn.com/i/teamlogos/ncaa/500/26...,34.161570,-118.167615,57.523380
115,2010,Ronald Powell,Florida,DE,5,"Moreno Valley, CA",33.9244,-117.2045,1,1,UNLV,#000000,['http://a.espncdn.com/i/teamlogos/ncaa/500/24...,36.085974,-115.017114,193.972416
116,2010,Ronald Powell,Florida,DE,5,"Moreno Valley, CA",33.9244,-117.2045,1,1,USC,#AE2531,['http://a.espncdn.com/i/teamlogos/ncaa/500/30...,34.014010,-118.287896,62.388128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473827,2020,John Young,Kentucky,OL,4,"Louisville, KY",38.1663,-85.6485,1,1,Ohio State,#DE3121,['http://a.espncdn.com/i/teamlogos/ncaa/500/19...,40.001686,-83.019728,189.610257
473836,2020,John Young,Kentucky,OL,4,"Louisville, KY",38.1663,-85.6485,1,1,Purdue,#B89D29,['http://a.espncdn.com/i/teamlogos/ncaa/500/25...,40.434323,-86.918485,170.778692
473850,2020,John Young,Kentucky,OL,4,"Louisville, KY",38.1663,-85.6485,1,1,Tennessee,#EE9627,['http://a.espncdn.com/i/teamlogos/ncaa/500/26...,35.954734,-83.925333,179.924069
473869,2020,John Young,Kentucky,OL,4,"Louisville, KY",38.1663,-85.6485,1,1,Vanderbilt,#000000,['http://a.espncdn.com/i/teamlogos/ncaa/500/23...,36.143559,-86.808772,153.666730


In [8]:
df['committedToSchool'] = np.where(df['committedTo'] == df['school'],1,0)

In [9]:
df = df.groupby('school').agg({'committedToSchool': ['sum', 'count']}).reset_index()

In [10]:
df.columns = ['school','committed','available']
df

,school,committed,available
0,Air Force,0,26
1,Akron,0,297
2,Alabama,91,602
3,Appalachian State,0,468
4,Arizona,4,55
...,...,...,...
126,West Virginia,12,424
127,Western Kentucky,0,422
128,Western Michigan,0,325
129,Wisconsin,11,106


In [11]:
df['%'] = 100 * df['committed'] / df['available']

In [12]:
df = df.sort_values(by = '%',ascending = False)

In [13]:
df.head(20)

,school,committed,available,%
124,Washington,25,60,41.666667
49,LSU,89,293,30.375427
68,Nebraska,12,43,27.906977
102,Texas,129,506,25.494071
115,USC,78,342,22.807018
78,Ohio State,74,345,21.449275
105,Texas Tech,3,15,20.000000
83,Oregon,10,53,18.867925
118,Utah,6,33,18.181818
34,Georgia,105,640,16.406250


In [14]:
df.to_csv('Blue Chips within 250 miles of Campus Hit Rate 2010-2020.csv')